In [1]:
import numpy as np
import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)



(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [40]:
def calc(a, w, b):
    return np.dot(w, a) + b

def sigmoid(x):
    return 1/(1+np.exp(-x))

def cost_function_derivative(a, y):
    return 2*(a-y)

def sigmoid_derivative(x):
    return (1/(1+np.exp(-x))**2)*np.exp(-x)

# def back():
    
digits = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
batch_size = 1
current = 0
learning_rate = 1
layers = 4

network = {1: {"activations" : x_train[current].reshape(784,1)/255}, 
            2: {"activations" : np.random.rand(16).reshape(16, 1), "weights" : np.random.uniform(-0.5, 0.5, size=784*16).reshape(16, 784), "biases" : np.random.uniform(-0.5, 0.5, size=16).reshape(16, 1)}, 
            3: {"activations" : np.random.rand(16).reshape(16, 1), "weights" : np.random.uniform(-0.5, 0.5, size=16*16).reshape(16, 16), "biases" : np.random.uniform(-0.5, 0.5, size=16).reshape(16, 1)},
            4 : {"activations" : np.random.rand(10).reshape(10, 1), "weights" : np.random.uniform(-0.5, 0.5, size=16*10).reshape(10, 16), "biases" : np.random.uniform(-0.5, 0.5, size=10).reshape(10, 1)}}


for i in range(50000):
    
    gradient = {2: {"weights" : np.random.uniform(-0.5, 0.5, size=784*16).reshape(16, 784), "biases" : np.random.uniform(-0.5, 0.5, size=16).reshape(16, 1)}, 
                3: {"weights" : np.random.uniform(-0.5, 0.5, size=16*16).reshape(16, 16), "biases" : np.random.uniform(-0.5, 0.5, size=16).reshape(16, 1)},
                4 : {"weights" : np.random.uniform(-0.5, 0.5, size=16*10).reshape(10, 16), "biases" : np.random.uniform(-0.5, 0.5, size=10).reshape(10, 1)}}
              
    
    z2 = calc(network[1]["activations"], network[2]["weights"], network[2]["biases"])
    network[2]["activations"] = sigmoid(z2)
    z3 = calc(network[2]["activations"], network[3]["weights"], network[3]["biases"])
    network[3]["activations"] = sigmoid(z3)
    z4 = calc(network[3]["activations"], network[4]["weights"], network[4]["biases"])
    network[4]["activations"] = sigmoid(z4)
    
    expect_num = y_train[current]
    expect_output = np.zeros(10)
    expect_output[digits.index(expect_num)] = 1
    
    output = network[4]["activations"]
    
    #back propagation

    #layer 4
    
    dc4 = cost_function_derivative(network[4]["activations"], expect_output.reshape((10,1)))
    da4 = sigmoid_derivative(z4)
    
    gradient[4]["weights"] = network[3]["activations"].reshape(-1)*dc4*da4
    gradient[4]["biases"] = dc4*da4 
    
    #layer 3
    
    da3 = sigmoid_derivative(z3)
    dc3 = np.sum(dc4*da4*network[4]["weights"], axis=0).reshape(16,1)
    gradient[3]["weights"] = network[2]["activations"].reshape(-1)*dc3*da3
    gradient[3]["biases"] = dc3*da3
    
    #layer 2
    
    da2 = sigmoid_derivative(z2)
    dc2 = np.sum(dc3*da3*network[3]["weights"], axis = 0).reshape(16, 1)
    gradient[2]["weights"] = network[1]["activations"].reshape(-1)*dc2*da2
    gradient[2]["biases"] = dc2*da2
    
    #gradient descent
    
    network[4]["weights"] -= learning_rate*gradient[4]["weights"]
    network[4]["biases"]  -= learning_rate*gradient[4]["biases"]
    network[3]["weights"] -= learning_rate*gradient[3]["weights"]
    network[3]["biases"]  -= learning_rate*gradient[3]["biases"]
    network[2]["weights"] -= learning_rate*gradient[2]["weights"]
    network[2]["biases"]  -= learning_rate*gradient[2]["biases"]
    
    
    current += 1
    network[1]["activations"] = x_train[current].reshape(784, 1)/255
    
current = 0
for i in range(5):
    
    network[1]["activations"] = x_test[current].reshape(784, 1)/255
    z2 = calc(network[1]["activations"], network[2]["weights"], network[2]["biases"])
    network[2]["activations"] = sigmoid(z2)
    z3 = calc(network[2]["activations"], network[3]["weights"], network[3]["biases"])
    network[3]["activations"] = sigmoid(z3)
    z4 = calc(network[3]["activations"], network[4]["weights"], network[4]["biases"])
    network[4]["activations"] = sigmoid(z4)
    
    expect_num = y_test[current]
    expect_output = np.zeros(10)
    expect_output[digits.index(expect_num)] = 1

    output = network[4]["activations"]
    
    percent = (output[expect_num]/np.sum(output))*100
    correct = 0
    print(percent)
    if(percent[0] > 80.0):
        correct +=1
    current +=1
print(correct)
print(correct/current)

[98.90145051]
[92.85944349]
[99.83220414]
[99.54226423]
[91.4746846]
1
0.2
